# Phase transitions

In [2]:
import os
from workflows.pyiron.build import bulk, repeat
from workflows.pyiron.thermodynamics import calculate_free_energy
import matplotlib.pyplot as plt
from pyiron_workflow import Workflow
from workflows.pyiron.kg import KnowledgeDict 

In [3]:
pair_style = "eam/alloy"
pair_coeff = "* * workflows/potentials/Fe_Ack.eam Fe"

In [4]:
kg = KnowledgeDict()

In [5]:
wf = Workflow('fe1')
wf.bulk = bulk('Fe', cubic=True, kg=kg)
wf.structure = repeat(wf.bulk, (5,5,5), kg=kg)
wf.free_energy = calculate_free_energy(wf.structure, 
                                  pair_style, pair_coeff,
                                  reference_phase='solid',
                                  temperature=100,
                                  mode='pscale',
                                  n_switching_steps=50000,
                                  pressure = [120000, 150000],
                                  cores=4, kg=kg)

In [6]:
wf.run()

/cmmc/ptmp/pyironhb/mambaforge/envs/pyiron_mpie_cmti_2025-11-03/lib/python3.12/site-packages/ase/io/lammpsdata.py:51: FutureWarning: "style" is deprecated; please use "atom_style".
  warnings.warn(


{'free_energy__results': {'free_energy': array([-3.42266636, -3.42266214, -3.42265792, ..., -3.21293381,
         -3.21292964, -3.21292546], shape=(50001,)),
  'error': 0.0,
  'reference_system': 0.01162107526928949,
  'einstein_crystal': 0.01238021454530465,
  'com_correction': -0.0007591392760151596,
  'work': -4.277510854232653,
  'pv': 0.8432234231215665,
  'unit': 'eV/atom',
  'temperature': 100,
  'pressure': array([120000. , 120000.6, 120001.2, ..., 149998.8, 149999.4, 150000. ],
        shape=(50001,))}}

In [7]:
kg.to_yaml('free_energy.yaml')